In [1]:
#load up the basics
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [3]:
#read in the master DF
ModelPredFile = '../data/tables/MasterDF_09162021.csv'
preds = pd.read_csv(ModelPredFile)
preds.head()

,Filename,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,archive,date,image,labeled,...,lr_lat,lr_lon,size,storm_id,ul_lat,ul_lon,ur_lat,ur_lon,Prediction6,Prediction7
0,Barry/20190716a_jpgs/jpgs/C19748935.jpg,0.093053,0.458411,0.679300,0.274590,0.021257,20190716a_jpgs,2019-07-16,C19748935.jpg,NaN,...,29.501893,-92.311318,4875221,barry,29.514684,-92.317575,29.509719,-92.306144,0.210064,0.520629
1,Barry/20190716a_jpgs/jpgs/C19748941.jpg,0.002945,0.449371,0.456580,0.237515,0.043771,20190716a_jpgs,2019-07-16,C19748941.jpg,NaN,...,29.506580,-92.308422,5842249,barry,29.518644,-92.316049,29.514679,-92.304272,0.144417,0.516580
2,Barry/20190716a_jpgs/jpgs/C19748947.jpg,0.002490,0.413135,0.469627,0.226322,0.052162,20190716a_jpgs,2019-07-16,C19748947.jpg,NaN,...,29.511428,-92.306184,6061687,barry,29.522755,-92.314927,29.519666,-92.302932,0.151956,0.512670
3,Barry/20190716a_jpgs/jpgs/C19748953.jpg,0.020509,0.361904,0.444304,0.202061,0.065795,20190716a_jpgs,2019-07-16,C19748953.jpg,NaN,...,29.516382,-92.305242,5873595,barry,29.527209,-92.314396,29.524401,-92.302525,0.156361,0.532304
4,Barry/20190716a_jpgs/jpgs/C19748959.jpg,0.043591,0.347595,0.443178,0.191845,0.061327,20190716a_jpgs,2019-07-16,C19748959.jpg,NaN,...,29.521040,-92.304028,6159050,barry,29.531420,-92.313768,29.529043,-92.301852,0.156329,0.540272


In [6]:
def MapDF(Storm,df):
    
    #select the storm
    StormSubset = df[df['Filename'].str.contains(str(Storm))]
    
    StormSubset["Latitude"] = (StormSubset["lr_lat"] + StormSubset["ur_lat"] + StormSubset["ll_lat"] + StormSubset["ul_lat"])/4
    StormSubset["Longitude"] = (StormSubset["lr_lon"] + StormSubset["ur_lon"] + StormSubset["ll_lon"] + StormSubset["ul_lon"])/4
    
    MapStep = StormSubset.rename({'Latitude': 'Lat','Longitude': 'Lon', 'Prediction7': 'WashPred' }, axis=1)
    
    MapStep["_id"] = MapStep["Lat"]*0
    
    SMAP = MapStep[["storm_id", "archive", "image", "date", "WashPred", "_id", "Lat", "Lon"]]
    
    return SMAP

In [7]:
SMAP = MapDF('Ida', preds)

<ipython-input-6-ac5ce646f19f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  StormSubset["Latitude"] = (StormSubset["lr_lat"] + StormSubset["ur_lat"] + StormSubset["ll_lat"] + StormSubset["ul_lat"])/4
<ipython-input-6-ac5ce646f19f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  StormSubset["Longitude"] = (StormSubset["lr_lon"] + StormSubset["ur_lon"] + StormSubset["ll_lon"] + StormSubset["ul_lon"])/4


In [9]:
SMAP.head()

,storm_id,archive,image,date,WashPred,_id,Lat,Lon
132287,ida,20210902a_jpgs,022207-0902211527418-RGB1.jpg,2021/09/02,0.000163,0.0,29.756996,-90.136326
132288,ida,20210902a_jpgs,022085-0902211459342-RGB1.jpg,2021/09/02,0.000017,0.0,29.853914,-90.821296
132289,ida,20210902a_jpgs,023219-0902211806471-RGB1.jpg,2021/09/02,0.000017,0.0,30.372990,-89.400284
132290,ida,20210902a_jpgs,022026-0902211449173-RGB2.jpg,2021/09/02,0.000552,0.0,29.609813,-90.687714
132291,ida,20210902a_jpgs,023207-0902211805491-RGB1.jpg,2021/09/02,0.000003,0.0,30.331740,-89.403231


In [13]:
# how many washover images?
predsWash = SMAP[SMAP["WashPred"] >= 0.5]
predsWash.shape

(0, 8)

In [10]:
SMAPcsv = '../data/tables/Ida_ml_pred.csv'

SMAP.to_csv(SMAPcsv, index=False)